## 提高爬虫的速度
- 并发 vs 并行
    - 并发(concurrency) 是指在一个时间段内发生若干时间的情况
    - 并行(parallelism) 是指在同一时刻发生若干时间的情况
- 同步 vs 异步
    - 同步：并发或并行的各个人物不是独立运行的，任务之间有一定的交替顺序，可能在运行完一个任务得到结果后，另一个任务才会开始运行
    - 异步：并发或并行的各个人物可以独立运行，一个任务的运行不受另一个任务影响，任务之间就象比赛的各个选手在不同的赛道比赛一样，跑步的速度不受其他赛道选手的影响。
    
    
    
多线程爬虫：运行在单核上的多线程爬虫，using 'Treading' & 'Queue'
多进程爬虫：运行在CPU的多核上，进程是并行的，using 'multiprocessing'
    两种方法，一种是 process + Queue, 一种是 Pool + Queue
多协程爬虫：Coroutine 

## 反爬虫 与 反反爬虫
1. 可以做一个 User-Agent pool, 并在爬虫过程中随意切换。
2. 在header 中写上 Host 和 Referer
3. 修改爬虫的间隔时间, 但通常固定的间隔时间并不像真正的用户，可以随机间隔时间。
4. 维护一个代理 IP pool (不一定稳定)

In [ ]:
# 随机间隔时间
import time
import random
sleep_time = random.randint(0,2)+random.random()
time.sleep(sleep_time)

## 登陆与验证码处理
1. 找到登陆输入框 inspector - select - <input> - name:'loginname',
    这个 loginname 就是要输入的 username 
2. 找到登陆输入框 inspector - select - <input> - name:'loginpass',
    这个 loginpass 就是要输入的 password
3. 查找 auto_login 的元素
3. 查找 hidden 元素并 copy their value



In [4]:
import requests
session = requests.session()

post_url = 'http://fanfou.com/'
agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:67.0) Gecko/20100101 Firefox/67.0'
headers = {
    'Host':'fanfou.com/',
    'User-Agent':agent
}
postdata = {
    'loginname':'avril_99@vii.im',
    'loginpass':'',
    'action':'/login',
    'token':'215816a5'
}
login = session.post(post_url,data = postdata, headers = headers)
#session.cookies.save()

print(login.status_code)

400


### 使用Cookies, 使登陆更简单

In [ ]:
import requests
import http.cookiejar as cookielib

In [ ]:
session = requests.session()
session.cookies = cookielib.LWPCookieJar(filename='cookies')
try:
    session.cookies.load(ignore_discard=True)
except:
    print("Cookie 未能加载")
    
def isLogin():
    url = "http://www.santostang.com/wp-admin/profile.php"
    login_code = session.get(url, headers=headers, allow_redirects=False).status_code
    if login_code == 200:
        return True
    else:
        return False
    
def login(secret, account):
    post_url = 'http://www.santostang.com/wp-login.php'
    postdata = {
        'pwd': secret,
        'log': account,
        'rememberme' : 'true',
        'redirect_to': 'http://www.santostang.com/wp-admin/',
        'testcookie' : 1,
    }
    try:
        # 不需要验证码直接登录成功
        login_page = session.post(post_url, data=postdata, headers=headers)
        login_code = login_page.text
        print(login_page.status_code)
        #print(login_code)
    except:
        pass
    session.cookies.save()

In [ ]:
if __name__ == '__main__':
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {
        "Host": "www.santostang.com",
        "Origin":"http://www.santostang.com",
        "Referer":"http://www.santostang.com/wp-login.php",
        'User-Agent': agent
    }
    if isLogin():
        print('您已经登录')

### 验证码
处理验证码的主要方式有
- 人手输入
- OCR 识别

In [ ]:
def get_captcha():
    #获取验证码图片所在的url
    r = session.get('http://www.santostang.com/wp-login.php?action=register', headers=headers)
    soup = BeautifulSoup(r.text, "lxml")
    captcha_url = soup.find("img", id="captcha_code_img")["src"]
    # 获取验证码图片
    r = session.get(captcha_url, headers=headers)
    with open('captcha.jpg', 'wb') as f:
        f.write(r.content)
        f.close()
    try:
        im = Image.open('captcha.jpg')
        im.show()
        im.close()
    except:
        print(u'请到 %s 目录找到captcha.jpg 手动输入' % os.path.abspath('captcha.jpg'))
    captcha = input("please input the captcha\n>")
    return captcha


In [ ]:
def register(account, email):      
    post_url = 'http://www.santostang.com/wp-login.php?action=register'
    postdata = {
        'user_login': account,
        'user_email': email,
        'redirect_to': '',
        }
    # 调用get_captcha函数，获取验证码数字
    postdata["ux_txt_captcha_challenge_field"] = get_captcha()  
    # 提交POST请求，进行注册
    register_page = session.post(post_url, data=postdata, headers=headers)
    # 若输出打印结果为200，则表示注册成功
    print(register_page.status_code)

In [ ]:


import requests
from bs4 import BeautifulSoup
import re
import os
from PIL import Image
if __name__ == '__main__':
    agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'
    headers = {
        "Host": "www.santostang.com",
        "Origin":"http://www.santostang.com",
        "Referer":"http://www.santostang.com/wp-login.php",
        'User-Agent': agent
    }
    session = requests.session()
    # 调用注册函数进行注册
    account = '18341432113' #改成自己用户名
    email = 'a12345@qq.com' # 改成自己邮箱
    register(account, email)



#### OCR 


In [ ]:
from PIL import Image
im = Image.open('captcha.jpg')
gray = im.convert('L')
gray.show()
gray.save("captcha_gray.jpg")

In [ ]:
threshold = 150
table = []
for i in range(256):
    if i < threshold:
        table.append(0)
    else:
        table.append(1)
out = gray.point(table, '1')
out.show()
out.save("captcha_thresholded.jpg")

In [ ]:
import pytesseract
th = Image.open('captcha_thresholded.jpg')
print(pytesseract.image_to_string(th))